# Machine Learning 2019 Project


In [1]:
import os
import sys
import math
import copy
import json
import argparse

In [2]:
#easier access when need to get the log value
def log(val):
    if val == 0:
        return - sys.maxint - 1
    return math.log(val)

In [3]:
# Process Data

def process_data_emission( filePath ):
    tags = {}  # save the number of times each tag appears
    observations = {}  # save the word observation
    labelled_observations = {}  # save labelled observations
    
    for line in open(filePath, 'r', encoding="utf8"):
        segmentedLine = line.rstrip()
        if segmentedLine:  # if its not just an empty string
            segmentedLine = segmentedLine.rsplit(' ', 1)

            observation = segmentedLine[0]  # X
            tag = segmentedLine[1]  # Y

            if observation not in observations:  # if this observation has never been seen before
                observations[observation] = 1
            else:  # if this observation has been seen before
                observations[observation] += 1

            if tag not in tags:  # if this tag has never been seen before
                tags[tag] = 1
                labelled_observations[tag] = {observation: 1}

            else:  # if this tag has been seen before
                tags[tag] += 1
                if observation not in labelled_observations[tag]:
                    labelled_observations[tag][observation] = 1
                else:
                    labelled_observations[tag][observation] += 1
    return observations, tags, labelled_observations


## Part 2: Estimate Emission

Write a function that estimates the emission parameters from the training set using MLE (maximumlikelihood estimation):

$ e(x|y) = Count(y→x)/Count(y) $

In [4]:
# Estimate the emission probability
# Check whether it is unkown or not

def estimateEmission(observations, tags, labelled_observations , k=3): # As speciffied in the requirements k = 3   
    estimates = {} # save the estimates for each 
    for tag in labelled_observations:
        estimates[tag] = {}
        labelled_observations[tag]['##UNK##'] = 0
        for observation in list(labelled_observations[tag]):  # loop over all keys in labelled_observations
            if observation == '##UNK##': 
                continue
            if observation not in observations:  # if this observation has been found to appear less than k times before
                labelled_observations[tag]['##UNK##'] += labelled_observations[tag].pop(observation)
            elif observations[observation] < k:  # if first meet an observation that appear less than k times
                labelled_observations[tag]['##UNK##'] += labelled_observations[tag].pop(observation)
                del observations[observation]
            else:  # compute the MLE for that emission
                estimates[tag][observation] = float(labelled_observations[tag][observation]) / tags[tag] ## MLE
        estimates[tag]['##UNK##'] = float(labelled_observations[tag]['##UNK##']) / tags[tag]

    return list(observations), estimates

In [5]:
# Create the sentiment analysis 
def sentimentAnalysis(inputPath, estimates, outputPath):
    f = open(outputPath, 'w', encoding="utf8") # specified to be written as dev.p2.out
    unkPrediction = ('##UNK##', 0.0)
    for tag in estimates:
        if estimates[tag]['##UNK##'] > unkPrediction[1]:
            unkPrediction = (tag, estimates[tag]['##UNK##'])
    for line in open(inputPath, 'r', encoding="utf8"):
        observation = line.rstrip()
        if observation:
            prediction = ('', 0.0)  # prediction is tuple of tag and the MLE of observation for the given tag
            for tag in estimates:
                if observation in estimates[tag] and estimates[tag][observation] > prediction[1]:
                    prediction = (tag, estimates[tag][observation])
            if prediction[0]:
                f.write('%s %s\n' % (observation, prediction[0]))
            else:
                f.write('%s %s\n' % (observation, unkPrediction[0]))
        else:
            f.write('\n')

    print('Finished writing to file %s' % (outputPath))
    return f.close()    


In [6]:
## For those who are running locally

# '''
# sample python run:
# python part2.py -d AL
# '''
# parser = argparse.ArgumentParser()
# parser.add_argument('-d', type=str, dest='dataset', help='Dataset to run script over', required=True)
# ## Parsing the Dataset

# args = parser.parse_args()

# trainFilePath = f'./Dataset/{str(args.dataset)}/{str(args.dataset)}/train'
# inputTestFilePath = f'./Dataset/{str(args.dataset)}/{str(args.dataset)}/dev.in'
# outputTestFilePath = f'./Dataset/{str(args.dataset)}/{str(args.dataset)}/dev.p2.out'
# observations, tags, labelled_observations = process_data( str(trainFilePath) )

# m_training, estimates = estimateEmission(observations, tags, labelled_observations)
# sentimentAnalysis(inputTestFilePath, estimates, outputTestFilePath)

In [7]:
## For those running in iPython Notebook
datasets = ['AL', 'CN', 'EN', 'SG']
for dataset in datasets:
    trainFilePath = f'./Dataset/{dataset}/{dataset}/train'
    inputTestFilePath = f'./Dataset/{dataset}/{dataset}/dev.in'
    outputTestFilePath = f'./Dataset/{dataset}/{dataset}/dev.p2.out'
    
    observations, tags, labelled_observations = process_data_emission( str(trainFilePath) )
    
    m_training, estimates = estimateEmission(observations, tags, labelled_observations)
    sentimentAnalysis(inputTestFilePath, estimates, outputTestFilePath)

Finished writing to file ./Dataset/AL/AL/dev.p2.out
Finished writing to file ./Dataset/CN/CN/dev.p2.out
Finished writing to file ./Dataset/EN/EN/dev.p2.out
Finished writing to file ./Dataset/SG/SG/dev.p2.out


## Part 3: Emission, Transmission Parameter and Viterbi algorithm

Transmission parameter: $ q(y_{i}|y_{i−1}) = Count(y_{i−1}, y_{i}) / Count(y_{i−1})$

Viterbi Algorithm: $ y^∗_{1}, . . . , y^∗_{n}= arg max_{y_{1},...,y_{n}}p(x_{1}, . . . , x_{n}, y_{1}, . . . , y_{n}) $

Use estimate emission and process data from the previous part for ease of use

In [8]:
def process_data_transmission( filePath ):
    tags = {}  # count the number of times a particular y(i-1) tag has appeared
    t_Tags = {}  # count the number of times a particular transition from y(i) to y(i-1) has appeared

    previousState = ''
    currentState = '##START##'
    
    # Process the data
    for line in open(filePath, 'r'):
        previousState = currentState if (currentState != '##STOP##') else '##START##'  # y(i-1)
        segmentedLine = line.rstrip()

        if segmentedLine:  # if its not just an empty string
            segmentedLine = segmentedLine.rsplit(' ', 1)
            currentState = segmentedLine[1]  # y(i)
        else:  # if an empty string is seen
            if previousState == '##START##': 
                break  # training data always terminates with 2 empty lines
            currentState = '##STOP##'  # y(i)

        if previousState not in tags:  # if tag y(i-1) has never been seen before
            tags[previousState] = 1
            t_Tags[previousState] = {currentState: 1}
        else:
            tags[previousState] += 1
            if currentState not in t_Tags[previousState]:
                t_Tags[previousState][currentState] = 1
            else:
                t_Tags[previousState][currentState] += 1
    return tags, t_Tags

In [9]:
## Estimate transition
def estimateTransition(tags, t_Tags):
    estimates = {}
    # Compute the MLE for transitions
    for tag in t_Tags:
        estimates[tag] = {}
        for transition in t_Tags[tag]:
            estimates[tag][transition] = float(t_Tags[tag][transition]) / tags[tag]
    return estimates

In [ ]:
# Implementation of Viterbi Algorithm: 
# Generally viterbi algorithm implements trans * emission until stop
# Affected by the previous values hence is very troublesome and computation heavy if we do not use DP
def viterbi(observationSequence, m_training, emissionEstimates, transitionEstimates):
    tags = list(emissionEstimates)
    pi = [{tag: [0.0, ''] for tag in list(emissionEstimates)} for o in observationSequence]
    
    # Initialization
    for c_tag in tags:
        if c_tag not in transitionEstimates['##START##']: continue  # update tags which can be transitioned from ##START##

        if observationSequence[0] in m_training:  # if this word is not ##UNK##
            if observationSequence[0] in emissionEstimates[c_tag]:  # and this emission can be found
                emission = emissionEstimates[c_tag][observationSequence[0]]
            else:  # but this emission doesn't exist
                emission = 0.0
        else:  # if this word is ##UNK##
            emission = emissionEstimates[c_tag]['##UNK##']

        pi[0][c_tag] = [transitionEstimates['##START##'][c_tag] * emission, '##START##']
    # Recursive case
    for k in range(1, len(observationSequence)):  # pi[k][c_tag] = max(a(p_tag, c_tag)...)
        for c_tag in tags:
            for p_tag in tags:
                if c_tag not in transitionEstimates[p_tag]: continue  # only compare p_tags which can transition to c_tag

                score = pi[k-1][p_tag][0] * transitionEstimates[p_tag][c_tag]
                if score > pi[k][c_tag][0]:
                    pi[k][c_tag] = [score, p_tag]

            if observationSequence[k] in m_training:  # if this word is not ##UNK##
                if observationSequence[k] in emissionEstimates[c_tag]:  # and this emission can be found
                    emission = emissionEstimates[c_tag][observationSequence[k]]
                else:  # but this emission doesn't exist
                    emission = 0.0
            else:  # if this word is ##UNK##
                emission = emissionEstimates[c_tag]['##UNK##']

            pi[k][c_tag][0] *= emission

    # Finally
    result = [0.0, '']
    for p_tag in tags:
        if '##STOP##' not in transitionEstimates[p_tag]: continue  # only compare p_tags which can transition to ##STOP##

        score = pi[-1][p_tag][0] * transitionEstimates[p_tag]['##STOP##']
        if score > result[0]:
            result = [score, p_tag]

    # Backtracking
    if not result[1]:  # for those weird cases where the final probability is 0
        return

    prediction = [result[1]]
    for k in reversed(range(len(observationSequence))):
        if k == 0: break  # skip ##START## tag
        prediction.insert(0, pi[k][prediction[0]][1])

    return prediction
    